In [1]:
import gym
import numpy as np
import random
from IPython.display import clear_output

# Configuración del entorno
env = gym.make("MountainCar-v0", render_mode=None)  # Cambiar render_mode="human" si quieres una ventana gráfica

# Discretización del espacio de estados
state_bins = [20, 20]  # Número de divisiones en cada dimensión del espacio de estados
position_space = np.linspace(-1.2, 0.6, state_bins[0])
velocity_space = np.linspace(-0.07, 0.07, state_bins[1])

def discretize_state(state):
    """Convierte un estado continuo en un estado discreto."""
    position, velocity = state
    position_idx = np.digitize(position, position_space) - 1
    velocity_idx = np.digitize(velocity, velocity_space) - 1
    return (position_idx, velocity_idx)

# Inicialización de la tabla Q
q_table = np.zeros((*state_bins, env.action_space.n))  # (20, 20, 3)

# Hiperparámetros
alpha = 0.1        # Tasa de aprendizaje
gamma = 0.99       # Factor de descuento
epsilon = 1.0      # Tasa de exploración inicial
epsilon_decay = 0.999  # Decaimiento de epsilon
min_epsilon = 0.01  # Valor mínimo de epsilon
episodes = 10000    # Número de episodios de entrenamiento

# Entrenamiento del agente
for episode in range(episodes):
    state, info = env.reset()
    state = discretize_state(state)  # Convertir estado continuo en discreto
    done = False
    total_reward = 0

    while not done:
        # Selección de acción: explorar o explotar
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()  # Explorar
        else:
            action = np.argmax(q_table[state])  # Explotar

        # Ejecutar acción
        next_state, reward, done, truncated, info = env.step(action)
        next_state = discretize_state(next_state)  # Discretizar el nuevo estado

        # Recompensa ajustada para motivar al agente a avanzar
        reward = next_state[0] / len(position_space)

        # Actualización de la tabla Q
        old_value = q_table[state + (action,)]
        next_max = np.max(q_table[next_state])
        q_table[state + (action,)] = old_value + alpha * (reward + gamma * next_max - old_value)

        # Actualizar estado y recompensa acumulada
        state = next_state
        total_reward += reward

    # Reducir epsilon gradualmente
    epsilon = max(min_epsilon, epsilon * epsilon_decay)

    # Mostrar progreso
    if episode % 1000 == 0:
        clear_output(wait=True)
        print(f"Episodio {episode}/{episodes} completado. Epsilon: {epsilon:.3f}")

print("Entrenamiento completado!")

# Evaluación del agente entrenado
test_episodes = 5
print("\nEjecutando el agente entrenado...")

for episode in range(test_episodes):
    state, info = env.reset()
    state = discretize_state(state)
    done = False
    total_reward = 0

    while not done:
        env.render()  # Mostrar el entorno en una ventana gráfica
        action = np.argmax(q_table[state])  # Elegir la mejor acción
        next_state, reward, done, truncated, info = env.step(action)
        next_state = discretize_state(next_state)
        state = next_state
        total_reward += reward

    print(f"Episodio {episode + 1}: Recompensa total = {total_reward}")

env.close()


Episodio 9000/10000 completado. Epsilon: 0.010
Entrenamiento completado!

Ejecutando el agente entrenado...
Episodio 1: Recompensa total = -192.0
Episodio 2: Recompensa total = -192.0
Episodio 3: Recompensa total = -211.0
Episodio 4: Recompensa total = -200.0
Episodio 5: Recompensa total = -207.0


c:\Users\hegoi\TheBridge\2409_dsft_thebridge_core\.venv\lib\site-packages\gym\envs\classic_control\mountain_car.py:171: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("MountainCar-v0", render_mode="rgb_array")
  gym.logger.warn(
